In [1]:
from pokemon import *
from pokemon_statistics import *
from simulator import runSim, runSimList
import sqlite3
import requests
import pickle

In [2]:
import pickle
f = open("movedex.pkl", "rb")
movedex = pickle.load(f)
f.close()

In [3]:
def genMoveCombos(pokes, side, hitAlly=True):
    poke1 = pokes[0]
    poke2 = pokes[1]
    
    base = ">" + side
    combos = []
    
    p1 = poke1.moves + poke1.pred_moves[:4 - len(poke1.moves)]
    p2 = poke2.moves + poke2.pred_moves[:4 - len(poke2.moves)]
    
    moves1 = []
    moves2 = []
    
    for m1 in p1:
        moves1.append(targetChoices(m1, ' -2', hitAlly))
        
    for m2 in p2:
        moves2.append(targetChoices(m2, ' -1', hitAlly))
        
    for i1 in range(len(p1)):
        m1 = base + ' move ' + str(i1 + 1)
        for t1 in moves1[i1]:
            m2 = m1 + t1
            for i2 in range(len(p2)):
                m3 = m2 + ', move ' + str(i2 + 1)
                for t2 in moves2[i2]:
                    m4 = m3 + t2 + '\n'
                    combos.append(m4)

    return combos
                    
def targetChoices(move, ally, hitAlly=True):
    v = movedex[move.lower()]
    
    if v == 'adjacentAlly':
        return [ally]
    if v == 'adjacentAllyOrSelf':
        return ['-1', '-2']
    if v == 'adjacentFoe':
        return [' 1', ' 2']
    if v == 'all':
        return ['  ']
    if v == 'allAdjacent':
        return ['  ']
    if v == 'allAdjacentFoes':
        return ['  ']
    if v == 'allies':
        return ['  ']
    if v == 'allySide':
        return ['  ']
    if v == 'allyTeam':
        return ['  ']
    if v == 'any':
        if hitAlly:
            return [ally, ' 1', ' 2']
        else:
            return [' 1', ' 2']
    if v == 'foeSide':
        return ['  ']
    if v == 'normal':
        if hitAlly:
            return [ally, ' 1', ' 2']
        else:
            return [' 1', ' 2']
    if v == 'randomNormal':
        return ['  ']
    if v == 'scripted':
        return ['  ']
    if v == 'self':
        return ['  ']
        

# Testing below this point

In [4]:
ms = state()
ms.team1 = team()
ms.team2 = team()

In [5]:
ms.team1.side = 'p1'
ms.team1.full = []
ms.team1.full.append(createPikalyticsPokemon('incineroar'))
ms.team1.full.append(createPikalyticsPokemon('regieleki'))
ms.team1.full.append(createPikalyticsPokemon('landorus-therian'))
ms.team1.full.append(createPikalyticsPokemon('rillaboom'))
ms.team1.full.append(createPikalyticsPokemon('urshifu-rapid-strike'))
ms.team1.full.append(createPikalyticsPokemon('porygon2'))

ms.team1.active.append(ms.team1.full[0])
ms.team1.active.append(ms.team1.full[1])

In [6]:
ms.team2.side = 'p2'
ms.team2.full = []
ms.team2.full.append(createPikalyticsPokemon('urshifu'))
ms.team2.full.append(createPikalyticsPokemon('grimmsnarl'))
ms.team2.full.append(createPikalyticsPokemon('torkoal'))
ms.team2.full.append(createPikalyticsPokemon('venusaur'))
ms.team2.full.append(createPikalyticsPokemon('thundurus'))
ms.team2.full.append(createPikalyticsPokemon('tapu fini'))

ms.team2.active.append(ms.team2.full[0])
ms.team2.active.append(ms.team2.full[1])

In [21]:
p1moves = genMoveCombos((ms.team1.active[0], ms.team1.active[1]), 'p1', False)
p2moves = genMoveCombos((ms.team2.active[0], ms.team2.active[1]), 'p2', False)
print(len(p1moves))
print(len(p2moves))
print(len(p2moves) * len(p1moves))

48
42
2016


In [8]:
%%time
round1 = runSimList(ms, p1moves, p2moves, side=1, sims_proc=30)
p1best = [x[0] for x in round1[:5]]
p1best

Best:	 ('>p1 move 1 2, move 3 1\n', 36.58191086113557)
CPU times: user 4.89 ms, sys: 144 ms, total: 148 ms
Wall time: 6.29 s


['>p1 move 1 2, move 3 1\n',
 '>p1 move 1 2, move 2 1\n',
 '>p1 move 3 2, move 2 1\n',
 '>p1 move 3 2, move 3 1\n',
 '>p1 move 2 2, move 2 1\n']

In [9]:
%%time
round2 = runSimList(ms, p1best, p2moves, side=2, sims_proc=40)
p2best = [x[0] for x in round2[:5]]
p2best

Best:	 ('>p2 move 1 2, move 1 1\n', 37.570297275865066)
CPU times: user 44.9 ms, sys: 137 ms, total: 182 ms
Wall time: 6.89 s


['>p2 move 1 2, move 1 1\n',
 '>p2 move 2 2, move 2  \n',
 '>p2 move 2 2, move 1 1\n',
 '>p2 move 1 2, move 4 1\n',
 '>p2 move 1 2, move 2  \n']

In [22]:
%%time
round3 = runSimList(ms, p1moves, p2best, side=1, sims_proc=50)
final_moves = [x[0] for x in round3[:5]]
final_moves

Best:	 ('>p1 move 3 2, move 3 1\n', 17.236229910356045)
CPU times: user 49.8 ms, sys: 212 ms, total: 262 ms
Wall time: 7.05 s


['>p1 move 3 2, move 3 1\n',
 '>p1 move 3 1, move 3 1\n',
 '>p1 move 1 2, move 3 1\n',
 '>p1 move 3 1, move 3 2\n',
 '>p1 move 1 1, move 3 2\n']

In [23]:
round3

[('>p1 move 3 2, move 3 1\n', 17.236229910356045),
 ('>p1 move 3 1, move 3 1\n', 16.446330559185636),
 ('>p1 move 1 2, move 3 1\n', 11.861476988537632),
 ('>p1 move 3 1, move 3 2\n', 5.09878779227546),
 ('>p1 move 1 1, move 3 2\n', 4.629483128783281),
 ('>p1 move 1 1, move 3 1\n', 2.5520949921527514),
 ('>p1 move 1 2, move 2 1\n', 2.1518834837230383),
 ('>p1 move 4 2, move 3 1\n', -1.5125368548960285),
 ('>p1 move 3 2, move 3 2\n', -5.628113283477246),
 ('>p1 move 2 1, move 3 1\n', -5.949979136575186),
 ('>p1 move 2 2, move 3 1\n', -6.3526056815014815),
 ('>p1 move 4 1, move 3 1\n', -7.43222871816036),
 ('>p1 move 1 2, move 3 2\n', -9.350064074182484),
 ('>p1 move 3 1, move 2 1\n', -13.119812916981914),
 ('>p1 move 1 1, move 1  \n', -13.390394354328905),
 ('>p1 move 4 2, move 2 1\n', -13.678488662677399),
 ('>p1 move 2 2, move 2 1\n', -14.829451409096286),
 ('>p1 move 2 1, move 2 1\n', -14.925116353715287),
 ('>p1 move 4 1, move 2 1\n', -16.11052387830433),
 ('>p1 move 1 1, move 2 2\n'

In [11]:
print(ms.team1.active[0].name)
print(ms.team1.active[0].pred_moves)

incineroar
['FlareBlitz', 'FakeOut', 'PartingShot', 'Taunt']


In [12]:
print(ms.team1.active[1].name)
print(ms.team1.active[1].pred_moves)

regieleki
['Electroweb', 'Thunderbolt', 'VoltSwitch', 'Protect']


In [13]:
print(ms.team2.active[0].name)
print(ms.team2.active[0].pred_moves)

urshifu
['CloseCombat', 'WickedBlow', 'SuckerPunch', 'Protect']


In [14]:
print(ms.team2.active[1].name)
print(ms.team2.active[1].pred_moves)

grimmsnarl
['SpiritBreak', 'LightScreen', 'Reflect', 'FakeTears']


In [15]:
1 = 2

SyntaxError: cannot assign to literal (<ipython-input-15-c0ab9e3898ea>, line 1)

In [ ]:
p1 = createPikalyticsPokemon('incineroar')
p2 = createPikalyticsPokemon('regieleki')

p3 = createPikalyticsPokemon('urshifu')
p4 = createPikalyticsPokemon('grimmsnarl')

combos = genMoveCombos([p1, p2], 'p1', False)
print(len(combos))

combos2 = genMoveCombos([p3, p4], 'p2', False)
print(len(combos2))

print(len(combos) * len(combos2))

combos

In [ ]:
def makeFormatString(poke_name, moves=[], maxmoves=[], ability='', item=''):
    # I should check the poke_name against table names in database or 
    # else ill get sql injected problem here, granted the consequences are small 
    # but lets be honest it also probably isnt
    # put in a validate statement like if(validate(poke_name))
        
    format_string = 'SELECT * FROM ' + poke_name 
    format_tuple = ()
    
    where = False
    
    for move in moves:
        if where:
            format_string += ' AND (move1=? OR move2=? OR move3=? OR move4=?)'
        else:
            format_string += ' WHERE (move1=? OR move2=? OR move3=? OR move4=?)'
            where = True
        format_tuple += (move, move, move, move)
    
    for move in maxmoves:
        if where:
            format_string += ' AND (maxmove1=? OR maxmove2=? OR maxmove3=? OR maxmove4=?)'
        else:
            format_string += ' WHERE (maxmove1=? OR maxmove2=? OR maxmove3=? OR maxmove4=?)'
            where = True
        format_tuple += (move, move, move, move)
                
    if ability_name != '':
        if where: 
            format_string += ' AND ability=?'
        else:
            format_string += ' WHERE ability=?'
            where = True
        format_tuple += (ability_name,)
        
    if item_name != '':
        if where: 
            format_string += ' AND item=?'
        else:
            format_string += ' WHERE item=?'
            where = True
        format_tuple += (item_name,)
    
    
    return format_string, format_tuple

def update_freq(freq, key):
    if key in freq:
        freq[key] += 1
    else:
        freq[key] = 1
        
def update_predictions(pokemon):
    format_string, format_tuple = makeFormatString(pokemon.name, pokemon.moves, pokemon.max_moves, 
                                                   pokemon.ability, pokemon.item)
    cursor.execute(format_string, format_tuple)
    
    total = 0
    move_freq = {}
    ability_freq = {}
    item_freq = {}
    for (num_id, ability, item, move1, move2, move3, move4, _, _, _, _) in cursor:
        total += 1

        update_freq(ability_freq, ability)
        update_freq(item_freq, item)

        update_freq(move_freq, move1)
        update_freq(move_freq, move2)
        update_freq(move_freq, move3)
        update_freq(move_freq, move4)
    
    if "UNKNOWN" in ability_freq.keys():
        ability_freq.pop("UNKNOWN")
    if "UNKNOWN" in item_freq.keys():
        item_freq.pop("UNKNOWN")
    if "UNKNOWN" in move_freq.keys():
        move_freq.pop("UNKNOWN")
    if "UNKNOWN" in maxmove_freq.keys():
        maxmove_freq.pop("UNKNOWN")
    
    if (pokemon.ability == ''):
        key, _ = max(ability_freq.items(), key=lambda x:x[1])
        pokemon.pred_ability = key
    
    if (pokemon.item == ''):
        key, _ = max(item_freq.items(), key=lambda x:x[1])
        pokemon.pred_item = key
    
    pokemon.pred_moves = []
    for i in range(4 - len(pokemon.moves)):
        key, _ = max(move_freq.items(), key=lambda x:x[1])
        pokemon.pred_moves.append(key)
        move_freq.pop(key)
    

In [ ]:
connection = sqlite3.connect("pokemon_profiles.db")
cursor = connection.cursor()